In [1]:
"""
Script train YOLOv8n để nhận diện biển báo giao thông
Tối ưu để deploy trên Raspberry Pi 5 (train trên GPU mạnh)
"""

import yaml
from ultralytics import YOLO
import torch
import os

In [2]:
print("\n💻 KIỂM TRA PHẦN CỨNG TRAINING")
print("=" * 50)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name())
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    print("✅ Sử dụng GPU để training nhanh hơn")
else:
    print("⚠️  Training trên CPU (sẽ chậm hơn)")


💻 KIỂM TRA PHẦN CỨNG TRAINING
CUDA available: True
GPU: NVIDIA H100 80GB HBM3
Memory: 84.9 GB
✅ Sử dụng GPU để training nhanh hơn


In [3]:
# Khởi tạo model YOLOv8n với pretrained weights
model = YOLO('yolov8n.pt')

# Cấu hình các hyperparameters
train_params = {
    'data': "dataset.yaml",          # File cấu hình dataset
    'epochs': 30,                   # Số epoch
    'patience': 15,                  # Early stopping
    'batch': -1,                     # Auto batch size
    'workers': 8,                   # Số worker
    'name': 'fpt_experiment',        # Tên experiment
    'exist_ok': True,                # Ghi đè nếu đã tồn tại
    'cache': True,                   # Cache dữ liệu
    'cos_lr': True,                  # Sử dụng cosine LR scheduler
    'close_mosaic': 10,              # Đóng mosaic sau 10 epoch
}

print("📊 Training parameters:")
for k, v in train_params.items():
    print(f"{k}: {v}")

100%|██████████| 6.25M/6.25M [00:00<00:00, 16.3MB/s]


📊 Training parameters:
data: dataset.yaml
epochs: 30
patience: 15
batch: -1
workers: 8
name: fpt_experiment
exist_ok: True
cache: True
cos_lr: True
close_mosaic: 10


In [4]:
print("\n=== 🔧 BẮT ĐẦU TRAINING ===")
results = model.train(**train_params)


=== 🔧 BẮT ĐẦU TRAINING ===
Ultralytics 8.3.170 🚀 Python-3.10.12 torch-2.7.1+cu126 CUDA:0 (NVIDIA H100 80GB HBM3, 80995MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=fpt_experiment, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=15, perspective=0.0, plo

100%|██████████| 5.35M/5.35M [00:00<00:00, 15.9MB/s]


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2492.0±791.6 MB/s, size: 35.5 KB)


train: Scanning /home/admin/archive/train/labels.cache... 1551 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1551/1551 [00:00<?, ?it/s]

train: /home/admin/archive/train/images/Poselet_10.jpg: 1 duplicate labels removed
train: /home/admin/archive/train/images/Poselet_180.jpg: 1 duplicate labels removed
train: /home/admin/archive/train/images/Poselet_247.jpg: 1 duplicate labels removed
train: /home/admin/archive/train/images/Poselet_268.jpg: 1 duplicate labels removed
train: /home/admin/archive/train/images/Poselet_282.jpg: 1 duplicate labels removed
train: /home/admin/archive/train/images/Poselet_290.jpg: 1 duplicate labels removed
train: /home/admin/archive/train/images/Poselet_298.jpg: 1 duplicate labels removed
train: /home/admin/archive/train/images/Poselet_359.jpg: 1 duplicate labels removed
train: /home/admin/archive/train/images/Skin_281.jpg: 1 duplicate labels removed
train: /home/admin/archive/train/images/VOC2010_723.jpg: 1 duplicate labels removed
train: /home/admin/archive/train/images/VOC2010_749.jpg: 1 duplicate labels removed
AutoBatch: Computing optimal batch size for imgsz=640 at 60.0% CUDA memory utili

...

      30/30        40G      1.206     0.9696      1.103        157        640: 100%|██████████| 5/5 [00:02<00:00,  2.13it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.26it/s]

                   all        510       1003      0.871       0.79      0.881      0.473



30 epochs completed in 0.035 hours.
Optimizer stripped from runs/detect/fpt_experiment/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/fpt_experiment/weights/best.pt, 6.2MB

Validating runs/detect/fpt_experiment/weights/best.pt...
Ultralytics 8.3.170 🚀 Python-3.10.12 torch-2.7.1+cu126 CUDA:0 (NVIDIA H100 80GB HBM3, 80995MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.06it/s]


                   all        510       1003      0.872      0.791      0.882      0.473
Speed: 0.0ms preprocess, 0.2ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/detect/fpt_experiment


In [5]:
best_model_path = f"{results.save_dir}/weights/best.pt"
print(f"✅ Best model saved at: {best_model_path}")

✅ Best model saved at: runs/detect/fpt_experiment/weights/best.pt


In [6]:
print("\n=== 📤 BẮT ĐẦU EXPORT ===")
try:
    export_path = model.export(format='ncnn')
    print(f"✅ Export thành công: {export_path}")
except Exception as e:
    export_path = None
    print(f"❌ Export thất bại: {e}")


=== 📤 BẮT ĐẦU EXPORT ===
Ultralytics 8.3.170 🚀 Python-3.10.12 torch-2.7.1+cu126 CPU (Intel Xeon Platinum 8462Y+)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from 'runs/detect/fpt_experiment/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (6.0 MB)

TorchScript: starting export with torch 2.7.1+cu126...
TorchScript: export success ✅ 0.7s, saved as 'runs/detect/fpt_experiment/weights/best.torchscript' (11.9 MB)

NCNN: starting export with NCNN 1.0.20250503...
NCNN: running '/home/admin/.local/lib/python3.10/site-packages/ultralytics/pnnx runs/detect/fpt_experiment/weights/best.torchscript ncnnparam=runs/detect/fpt_experiment/weights/best_ncnn_model/model.ncnn.param ncnnbin=runs/detect/fpt_experiment/weights/best_ncnn_model/model.ncnn.bin ncnnpy=runs/detect/fpt_experiment/weights/best_ncnn_model/model_ncnn.py pnnxparam=runs/detect/fpt_experiment/weights/best_ncnn_model/model.pnnx.param pnnxbin=

pnnxparam = runs/detect/fpt_experiment/weights/best_ncnn_model/model.pnnx.param
pnnxbin = runs/detect/fpt_experiment/weights/best_ncnn_model/model.pnnx.bin
pnnxpy = runs/detect/fpt_experiment/weights/best_ncnn_model/model_pnnx.py
pnnxonnx = runs/detect/fpt_experiment/weights/best_ncnn_model/model.pnnx.onnx
ncnnparam = runs/detect/fpt_experiment/weights/best_ncnn_model/model.ncnn.param
ncnnbin = runs/detect/fpt_experiment/weights/best_ncnn_model/model.ncnn.bin
ncnnpy = runs/detect/fpt_experiment/weights/best_ncnn_model/model_ncnn.py
fp16 = 0
optlevel = 2
device = cpu
inputshape = [1,3,640,640]f32
inputshape2 = 
customop = 
moduleop = 
get inputshape from traced inputs
inputshape = [1,3,640,640]f32
############# pass_level0
inline module = ultralytics.nn.modules.block.Bottleneck
inline module = ultralytics.nn.modules.block.C2f
inline module = ultralytics.nn.modules.block.DFL
inline module = ultralytics.nn.modules.block.SPPF
inline module = ultralytics.nn.modules.conv.Concat
inline module

NCNN: export success ✅ 1.1s, saved as 'runs/detect/fpt_experiment/weights/best_ncnn_model' (11.6 MB)

Export complete (2.1s)
Results saved to /home/admin/runs/detect/fpt_experiment/weights
Predict:         yolo predict task=detect model=runs/detect/fpt_experiment/weights/best_ncnn_model imgsz=640  
Validate:        yolo val task=detect model=runs/detect/fpt_experiment/weights/best_ncnn_model imgsz=640 data=dataset.yaml  
Visualize:       https://netron.app
✅ Export thành công: runs/detect/fpt_experiment/weights/best_ncnn_model


############# pass_level1
############# pass_level2
############# pass_level3
############# pass_level4
############# pass_level5
############# pass_ncnn


In [ ]:
# Lấy đường dẫn đến best model
best_model_path = os.path.join('runs', 'detect', 'fpt_hackathon', 'weights', 'best.pt')

if export_path and os.path.exists(best_model_path):
    print("\n🎉 HOÀN TẤT QUÁ TRÌNH TRAIN & EXPORT!")
    print(f"📁 PyTorch model: {best_model_path}")
    print(f"📱 NCNN model directory: {export_path}")
    size_mb = os.path.getsize(best_model_path) / (1024*1024)
    print(f"📊 Kích thước model (PyTorch): {size_mb:.2f} MB")
    
    # Kiểm tra kích thước NCNN model files
    ncnn_param_path = os.path.join(export_path, 'model.ncnn.param')
    ncnn_bin_path = os.path.join(export_path, 'model.ncnn.bin')
    
    if os.path.exists(ncnn_param_path) and os.path.exists(ncnn_bin_path):
        param_size_mb = os.path.getsize(ncnn_param_path) / (1024*1024)
        bin_size_mb = os.path.getsize(ncnn_bin_path) / (1024*1024)
        total_ncnn_size = param_size_mb + bin_size_mb
        print(f"📊 Kích thước NCNN param: {param_size_mb:.2f} MB")
        print(f"📊 Kích thước NCNN bin: {bin_size_mb:.2f} MB")
        print(f"📊 Tổng kích thước NCNN: {total_ncnn_size:.2f} MB")
    else:
        print("⚠️ Không tìm thấy file NCNN model")
else:
    print("⚠️ Không export được model sang NCNN hoặc không tìm thấy best model.")


🎉 HOÀN TẤT QUÁ TRÌNH TRAIN & EXPORT!
📁 PyTorch model: runs/detect/fpt_experiment/weights/best.pt
📱 NCNN model: runs/detect/fpt_experiment/weights/best_ncnn_model
📊 Kích thước model (PyTorch): 5.96 MB


In [ ]:
import zipfile
import os

# Danh sách file/thư mục cần zip
items_to_zip = [
    "archive",
    "archive.zip",
    "dataset.yaml",
    "runs",
    "train_yolo.ipynb",
    "yolo11n.pt",
    "yolov8n.pt"
]

# Tên file zip đầu ra
output_zip = "selected_workspace.zip"

# Tạo file zip mới
with zipfile.ZipFile(output_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for item in items_to_zip:
        if os.path.exists(item):
            if os.path.isdir(item):
                # Duyệt và thêm từng file trong thư mục
                for root, _, files in os.walk(item):
                    for file in files:
                        filepath = os.path.join(root, file)
                        arcname = os.path.relpath(filepath, ".")
                        zipf.write(filepath, arcname)
            else:
                zipf.write(item)
        else:
            print(f"⚠️ Không tìm thấy: {item}")

print(f"✅ Đã tạo file ZIP: {output_zip}")
